# Advanced Data Processing with Pandas

We have learned many useful Pandas functions in the previous tutorials. 

In this tutorial, we will apply everything we have learned to a more challenging data science problem. Instead of just using the dataset and assuming it is "correct", we will have to combine data from different sources and clean it up to make it useful for our analysis.

**The Task**
- We will expand our original IMDB dataset to include more movies, from a different `csv` file.
- We will have to analyse both data sources to find out how to combine them.
- We will explore our new dataset and find if there are any missing or duplicated values.

# Getting Started

Before we start using the new dataset, let's load the original one again and learn a few more functions that will be useful for our task.

In [ ]:
import pandas as pd

movies_df_original = pd.read_csv("datasets/IMDB-Movie-Data.csv")

In [ ]:
movies_df_original.head(5)

# Finding Missing Values

In the previous tutorials, we noticed that some values in our dataset were NaN (Not a Number). This is a special value that Pandas uses to indicate that a value is missing.

We can systematically check for missing values in a DataFrame using the `isna()` function. This function returns a DataFrame of the same size as the original, but with boolean values. A value is `True` if the original value was NaN, and `False` otherwise.

Let's see how it works in practice.

In [ ]:
movies_df_original.isna()

The function works on a row-level. We can aggregate the results to check which columns have missing values.

How do we aggregate the results?

### `any()`

The `any()` functions returns `True` if any (i.e., at least one) of the values in the row are `True`. 

In [ ]:
movies_df_original.isna().any()

So we know that `Revenue (Millions)` and `Metascore` have missing values. But how many are missing in total?

### `sum()`

The `sum()` function returns the sum of all the values in a row. For boolean values, `True` is treated as `1` and `False` as `0`. So we can use `sum()` to count the number of `True` values in a row.

In [ ]:
movies_df_original.isna().sum()

# Handling Missing Values

We have seen that there are missing values in our dataset. But how do we deal with them?

It is important to think about what a *missing value* means in the context of our dataset. For instance, if a Metascore is missing, it could mean that the movie was not rated by the Metacritic website.  If a Revenue value is missing, it could mean that the movie was not released yet or simply that the information is not available.

With that in mind, we have two main options to handle missing values: removing or filling them. 

The best one depends on the situation and on the dataset. 

### Option 1: Remove the rows with missing values

We can assume that if a value is missing, the entire row is not useful for our analysis. If we are analysing the average Metascore per genre and a movie does not have this value, it is not useful to consider it for this.

We can remove the rows with missing values using the `dropna()` function.

`dropna()` has a `how` parameter that can be set to `any` or `all`. The default value is `any`, which means that a row will be removed if any of its values are missing. If we set it to `all`, a row will be removed only if all of its values are missing.

Let's try it out and remove all the rows with any missing values.

In [ ]:
movies_df_original.dropna(how="any")

If we remove all rows with any missing values, we drop 162 rows. *How do we calculate this?*

We can also remove all rows with missing values in specific columns. For that, we can use the `subset` parameter and pass a list of column names.

Let's remove all rows with missing values in the `Revenue (Millions)` column.

In [ ]:
movies_df_original.dropna(subset=["Revenue (Millions)"], how="any")

In this case, we drop 128 rows.

Remember that unless we set `inplace=True`, we are not modifying the original DataFrame. So we either need to assign the result to a new variable or set `inplace=True`.

### Option 2: Fill the missing values

Another option is to fill the missing values with some other value. This is useful if we want to keep all rows.

There are different ways to fill the missing values. We can fill them with a constant value, or we can fill them with the mean, median or mode of the column.

Again, the best option depends on the situation and on the dataset.

The `fillna()` function can be used to fill the missing values. It has a `value` parameter that can be set to a constant value, or to a function that will be applied to the column.

We can apply it directly to a specific column.

Let's fill the missing values in the `Metascore` column with the mean value.



In [ ]:
movies_df_original["Metascore"].fillna(movies_df_original["Metascore"].mean())

#### Task: check the descriptive statistics of the `Metascore` column before and after filling the missing values. What happened?

Check it again after filling the missing values with 0.

#### Task: do the same for the `Revenue (Millions)` column.

You can also apply `fillna()` to the entire DataFrame. In this case, the function will fill the missing values in all columns.

You should be careful when using this option, because it will fill all missing values, even if they are not supposed to be filled. For instance, if we fill the missing values in the `Genre` column, we will replace the missing values with the mean value of the column, which is not correct.

# Handling Duplicate Values

We can also check if there are duplicate values in our dataset. Duplicate values are rows that have the same values in all (or a subset of) columns.

If different rows have the exact same values in all columns, it is safe to assume they are duplicates. But if different rows have the same values in some columns, it is not clear if they are duplicates or not. For instance, if two movies have the same title, they could still be different. However, in our dataset, this is very unlikely.

To check for duplicate values, we can use the `duplicated()` function. This function returns a Series of boolean values, with one value for each row. A value is `True` if the row is a duplicate, and `False` otherwise. It works very similarly to `isna()`.

Let's check if there are any duplicate values in our dataset.

In [ ]:
movies_df_original.duplicated().sum()

There are no duplicate values in our dataset. But let's test it for one column just to see how it works.

Let's check if there are any duplicate (Director, Year) pairs.

In [ ]:
movies_df_original.duplicated(subset=["Director", "Year"]).sum()

That means 13 rows have the same Director and Year as another row. Of course they are not duplicates, because they are different movies, so we don't need to remove them.

But in case we wanted to remove them, we could use the `drop_duplicates()` function.

It works very similarly to `dropna()`. We can use the `subset` parameter to specify which columns to consider when checking for duplicates.

If we had duplicates in our dataset, we could remove them like this:

In [ ]:
movies_df_original.drop_duplicates(subset=["Director", "Year"], keep="first")

The `keep` parameter can be set to `first` or `last`. If we set it to `first`, the first row will be kept and the rest will be removed. If we set it to `last`, the last row will be kept and the rest will be removed.

# Combining Datasets

It's time to use our new dataset! Let's first load it and take a look at it.

In [ ]:
movies_df_new = pd.read_csv("datasets/imdb_5k.csv")
movies_df_new.head(5)

The columns are completely different! So we can't just combine the two datasets. We need to decide which columns to keep and which to remove.
Now it's time to work as real data scientist and investigate the data carefully.

#### Task: have a look at the columns in the new dataset and figure out what they mean.

In [ ]:
print(movies_df_original.columns)
print(movies_df_new.columns)

### Task: select at least 5 columns that are present in both datasets.

## Concatenating Datasets

If we want to simply add one dataset to the other, we can use the `concat()` function.

In this case, Pandas will just add the rows of the second dataset to the end of the first one.

Let's see how the output looks like.

In [ ]:
movies_df_combined = pd.concat([movies_df_original, movies_df_new])
movies_df_combined.tail(5)

That doesn't look good... Since the columns don't match, Pandas added a lot of missing values.

Let's do two things:
- Decide which columns to keep and rename them to a common name (using the `rename()` function
- Concatenate the two datasets again

In [ ]:
# Columns from the original dataframe. Let's keep using these names and rename the corresponding columns in the new dataframe.
columns_to_keep = ["Title", "Year", "Director", "Rating", "Revenue (Millions)"]
rename_mapping = {"movie_title": "Title", "title_year": "Year", "director_name": "Director", "imdb_score": "Rating", "gross": "Revenue (Millions)"}
# Now let's rename the columns in the new dataframe.
movies_df_new.rename(columns=rename_mapping, inplace=True)
# Now let's concatenate the dataframes using only the columns we want to keep.
movies_df_combined = pd.concat([movies_df_original[columns_to_keep], movies_df_new[columns_to_keep]])
movies_df_combined.tail(5)

It looks much better now!

However, we might have added some duplicate rows. The new dataset might have movies that are already in the old dataset.

#### Task: check if there are any duplicate rows in the new dataset and remove them.

This was not the only problem...

#### Task: investigate the combined dataset and figure out at least three other issues we need to fix.

**Hint:** have a look at the index.

### Fixing potential issues

We will do three things:
- Fix the index
- Fix the column types
- Handle missing values

#### Fixing the index

The index is not correct. It should be unique for each row, but it is not. We can fix it by resetting it.

The `reset_index()` function will reset the index and create a new column with the old index values, in case we need to use them later.

In [ ]:
movies_df_combined.reset_index(inplace=True)
movies_df_combined.tail(5)

#### Fixing the column types

In our original dataset, the `Year` column was an integer. But in the new dataset, it is a float. When we concatenated the two datasets, Pandas converted the column to a float.

However, it makes more sense to keep it as an integer. We can convert it using the `astype()` function.

In [ ]:
movies_df_combined["Year"] = movies_df_combined["Year"].astype(int)
movies_df_combined.head(5)

That didn't work... Why?

Some years are missing, and we can't convert NaN to an integer.

We need first handle the missing values. Then we come back to this.

#### Handling missing values

We already saw that the missing values in the `Year` column are not a critical. In this case, it makes sense to remove them.

Let's do that.

In [ ]:
movies_df_combined.dropna(subset=["Year"], how="any", inplace=True)

Now let's try to convert the column to an integer again.

In [ ]:
movies_df_combined["Year"] = movies_df_combined["Year"].astype(int)
movies_df_combined.head(5)

Now it works!

#### Task: check what other columns have missing values and decide what to do with them.

In [ ]:
movies_df_combined

Now we have a nice combined dataset!

Time to answer the quiz.

# !!! QUIZ !!!

# Adding more Information

There are more columns that are present in both datasets. Let's add at least genre and runtime.

#### Task: map the genre and runtime columns from the old dataset to the new one. 

Use the same approach as before and rename the columns to a common name.

#### Task: create a new combined dataset including the seven columns we selected.

In [ ]:
# Columns from the original dataframe. Let's keep using these names and rename the corresponding columns in the new dataframe.
columns_to_keep = ["Title", "Year", "Director", "Rating", "Revenue (Millions)", "Genre", "Runtime (Minutes)"]
rename_mapping = {"movie_title": "Title", "title_year": "Year", "director_name": "Director", "imdb_score": "Rating", "gross": "Revenue (Millions)", "genres": "Genre", "duration": "Runtime (Minutes)"}
# Now let's rename the columns in the new dataframe.
movies_df_new.rename(columns=rename_mapping, inplace=True)
# Now let's concatenate the dataframes using only the columns we want to keep.
movies_df_combined = pd.concat([movies_df_original[columns_to_keep], movies_df_new[columns_to_keep]])
movies_df_combined.tail(5)

#### Task: Fix the index and the runtime colum (the values should be integers).

In [ ]:
movies_df_combined.reset_index(inplace=True)
movies_df_combined.dropna(subset=["Year"], how="any", inplace=True)
movies_df_combined.dropna(subset=["Runtime (Minutes)"], how="any", inplace=True)
movies_df_combined["Runtime (Minutes)"] = movies_df_combined["Runtime (Minutes)"].astype(int)
movies_df_combined.tail(5)

#### Task: fix the genre column. The values should be separated by commas.

In [ ]:
movies_df_combined["Genre"] = movies_df_combined["Genre"].str.replace("|", ",")
movies_df_combined.tail(5)

# Bonus Tasks

Using the most recent dataset, answer the following questions:

### Find out which directors were included in the combined dataset.

What is their average IMDB rating?

### What is the average runtime of movies released in the last 10 years?

### Are movies getting shorter? Plot the average runtime of movies released in each year.

### Compare the median runtime of Documentaries and Comedy movies.

### Which director directed the most movies with different genres?

### What year had the highest average IMDB rating? And the lowest?